In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


In [ ]:
# Setting random seed for reproducibility
torch.manual_seed(140)
np.random.seed(140)
random.seed(140)

In [ ]:

"""
# Load data
x_values = torch.tensor([1, 2, 3]) # X values
y_values = torch.tensor([20, 30, 40]) # Y values

x_values_np = x_values.numpy()
y_values_np = y_values.numpy()

# Plot data
plt.plot(x_values_np, y_values_np)
plt.xlabel('Time')
plt.ylabel('Values')
plt.title('Values over time')
plt.show()

"""


In [ ]:
# Simulate data 
x_values = torch.tensor([1, 2, 3], dtype=torch.float32).view(-1, 1, 1) # X values
y_values = x_values * 10 # Y values

# Print shapes
print(x_values.shape, x_values.shape)

In [ ]:
# Define the LSTM model
class SimpleLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=10, num_layers=1, batch_first=True)
        self.linear = nn.Linear(10, 1)
    
    def forward(self, x):
        x, _ = self.lstm(x)
        # Extract only the last timestep's output for prediction
       # x = x[:, -1, :]
        x = self.linear(x[:, -1, :])
        return x
    
# Define the GRU model
class SimpleGRU(nn.Module):
    def __init__(self):
        super().__init__()
        self.gru = nn.GRU(input_size=1, hidden_size=10, num_layers=1, batch_first=True)
        self.linear = nn.Linear(10, 1)
    
    def forward(self, x):
        x, _ = self.gru(x)
        # Extract only the last timestep's output for prediction
       # x = x[:, -1, :]
        x = self.linear(x[:, -1, :])
        return x 

In [ ]:
# Initialize the model
model = SimpleLSTM()
#model = SimpleGRU()
optimizer = optim.SGD(model.parameters(), lr=0.05)
loss_fn = nn.MSELoss()


# Train the model
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(x_values)
    loss = loss_fn(output, y_values.view(-1, 1))
    loss.backward()
    optimizer.step()
    
    if epoch % 5 == 0:
        print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item()}')


model.eval()
with torch.no_grad():
    predictions = model(x_values) 
    print(f"Predictions: {predictions.view(-1).numpy()}")
    print(f"Actual: {y_values.view(-1).numpy()}")

# Visualizing the results
plt.figure(figsize=(10, 5))
plt.plot(x_values.numpy().flatten(), y_values.numpy().flatten(), 'bo-', label='Actual')
plt.plot(x_values.numpy().flatten(), predictions.numpy().flatten(), 'ro-', label='Predicted')
plt.title('LSTM Predictions vs Actual')
plt.xlabel('X Value')
plt.ylabel('Y Value')
plt.legend()


